In [5]:
import findspark
from pyspark.sql import SparkSession
import schemas
from test import most_used_tags

findspark.init()


def init_spark():
    spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
    sc = spark.sparkContext
    return spark, sc


def main():
    spark, sc = init_spark()
    timestamp_format = 'YYYY-MM-DD HH:mm:ss z'
    dummy_switch = '-1.4.0-2018-12-22'
    # dummy_switch = '_dummy'

    projects_path = f'Libraries.io-open-data-1.4.0/libraries/projects{dummy_switch}.csv'
    projects = spark.read.csv(projects_path, header=True, timestampFormat=timestamp_format,
                          schema=schemas.projects)
    most_used_tags(projects)
    # nums = sc.parallelize([1, 2, 3, 4])
    # print(nums.map(lambda x: x * x).collect())
    # dist_file = sc.textFile('Libraries.io-open-data-1.4.0.tar.gz', 5000)
    # size = dist_file.map(lambda s: len(s)).reduce(lambda a, b: a+b)
    # print(size)


if __name__ == '__main__':
    main()



[Row(Keyword='react', count=36788), Row(Keyword='api', count=24728), Row(Keyword='javascript', count=23676), Row(Keyword='library', count=21238), Row(Keyword='laravel', count=20495), Row(Keyword='framework', count=19733), Row(Keyword='angular', count=18990), Row(Keyword='plugin', count=18054), Row(Keyword='test', count=17573), Row(Keyword='css', count=16400)]


In [19]:
spark, sc = init_spark()
import pyspark.sql.functions as F
dependencies_path = f'Libraries.io-open-data-1.4.0/libraries/dependencies-1.4.0-2018-12-22.csv'
projects_path = f'Libraries.io-open-data-1.4.0/libraries/projects-1.4.0-2018-12-22.csv'
timestamp_format = 'YYYY-MM-DD HH:mm:ss z'

dependencies = spark.read.csv(projects_path, header=True, schema=schemas.projects)
dependencies = spark.read.csv(dependencies_path, header=True, schema=schemas.dependencies).drop('ID').select('Project ID', 'Dependency Project ID')
dependency_pairs = dependencies.where(F.col('Dependency Project ID').isNotNull()).limit(10)
pageranks = dependency_pairs \
    .select('Project ID') \
    .union(dependency_pairs
           .select('Dependency Project ID')) \
    .distinct().alias('ID') \
    .withColumn('pagerank', F.lit(1))
dependency_counts = dependency_pairs.groupBy('Project ID').count()
# print(pageranks.count(), pageranks.take(5))
for it in range(2):
    print(f'It {it}')
    dependency_pairs = pageranks \
        .join(dependency_pairs, 'Project ID') \
        .join(dependency_counts, 'Project ID') \
        .withColumn('vote', F.col('pagerank') / F.col('count'))\
        .drop('pagerank', 'count')
    vote_sums = dependency_pairs\
        .groupBy('Dependency Project ID')\
        .sum('vote')\
        .withColumnRenamed('sum(vote)', 'vote_sum')
    # print(vote_sums.take(5))
    pageranks = pageranks\
        .join(vote_sums, pageranks['Project ID'] == vote_sums['Dependency Project ID'], how='left')\
        .drop('pagerank')\
        .drop('Dependency Project ID')\
        .withColumn('pagerank', 1 + F.col('vote_sum'))\
        .drop('vote_sum') \
        .fillna({'pagerank': 1})
    pageranks.show(10)
pageranks.join(projects.select('ID', 'SourceRank', pageranks['Project ID']==projects['ID'])).drop('ID').show(10)

It 0


Py4JJavaError: An error occurred while calling o499.showString.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:150)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.